In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
import tensorflow_hub as hub
import logging
import re
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore') 
import keras        

In [2]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns',8000)
pd.set_option('display.max_rows',7000)

In [3]:
from google.colab import files
files.upload()

Saving embold_test.json to embold_test.json
Saving embold_train.json to embold_train.json


In [5]:
test  = pd.read_json('embold_test.json')
train_extra = pd.read_json('embold_train.json')

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [6]:
test.head()

,title,body
0,config question path-specific environment var...,issue description or question\r \r hey @artemg...
1,crash indien vol,de simulator crasht als hij vol zit
2,unable to mine rocks,"sarkasmo starting today, when i hit enter act..."
3,not all whitelists are processed,create following rules... order of creation is...
4,add ctx menu for idafree 70 and idafree 5,"associated with .dll, .dll_, .exe, .exe_, .sc,..."


In [7]:
train_extra.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [8]:
print(test.shape)
print(train_extra.shape)

(30000, 2)
(150000, 3)


In [9]:
train_extra['total']=train_extra['title']+train_extra['body']
test['total']=test['title']+test['body']

In [14]:
test.head()

,total
0,config question path-specific environment var...
1,crash indien volde simulator crasht als hij vo...
2,"unable to mine rockssarkasmo starting today, w..."
3,not all whitelists are processedcreate followi...
4,add ctx menu for idafree 70 and idafree 5assoc...


In [19]:
train_extra.head()

,label,total,clean_total,total_new
0,1,y-zoom piano rolla y-zoom on the piano roll wo...,zoom piano rollazoom the piano roll would useful,zoom piano rollazoom the piano roll would useful
1,0,buggy behavior in selection! screenshot from 2...,buggy behavior selectionscreenshot from,buggy behavior selectionscreenshot from
2,1,"auto update featurehi,\r \r great job so far, ...",auto update featurehi great job far saenzramir...,auto update featurehi great job far saenzramir...
3,1,filter out noisy endpoints in logsi think we s...,filter out noisy endpoints logsi think should ...,filter out noisy endpoints logsi think should ...
4,0,enable pid on / pid off alarm actions for ardu...,enable pid pid off alarm actions for arduino t...,enable pid pid off alarm actions for arduino t...


In [12]:
train_extra=train_extra.drop(labels=['title','body'],axis=1)

test=test.drop(labels=['title','body'],axis=1)

In [15]:
def cleaning(Inpdata):
    cleanedArticle1=re.sub(r'\\ruser[?|$|(),"".@#=><|!]Â&*/-',r' ',Inpdata)
    cleanedArticle2=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle1)
    cleanedArticle3=re.sub(r'\b\w{1,2}\b', ' ',cleanedArticle2)
    cleanedArticle4=re.sub(r' +', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle4)
    cleanedArticle6=cleanedArticle5.replace("  ", "")
    cleanedArticle7=cleanedArticle6.lower()
    return(cleanedArticle7)

In [16]:
train_extra['clean_total']=train_extra['total'].apply(cleaning)
test['clean_total']=test['total'].apply(cleaning)

In [18]:
train_extra['total_new'][8000]

'fixforbiddenbot was blocked the usertelegram error unauthorizedforbiddenbot was blocked the user'

In [17]:
train_extra['total_new']=train_extra['clean_total'].drop_duplicates()
test['total_new']=test['clean_total'].drop_duplicates()

In [20]:
train_extra=train_extra.drop(labels=['clean_total','total'],axis=1)

test=test.drop(labels=['clean_total','total'],axis=1)

In [21]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [24]:
import tensorflow_hub as hub
from  tokenization import FullTokenizer
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2, Total size: 421.50MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'.


In [25]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [26]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [27]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(3, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [30]:
max_len = 150
train_input = bert_encode(train_extra['total_new'].values, tokenizer, max_len=max_len)
test_input = bert_encode(test['total_new'].values, tokenizer, max_len=max_len)


In [29]:
train_extra=train_extra.dropna()

In [31]:
train_labels = tf.keras.utils.to_categorical(train_extra.label.values, num_classes=3)

In [32]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [33]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

In [34]:
train_history = model.fit(train_input, train_labels, validation_split=0.2,epochs=1,callbacks=[checkpoint, earlystopping],batch_size=32,
    verbose=1)

3750/3750 [==============================] - 7154s 2s/step - loss: 0.6323 - accuracy: 0.7552 - val_loss: 0.5091 - val_accuracy: 0.8105

Epoch 00001: val_accuracy improved from -inf to 0.81048, saving model to model.h5


In [36]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [39]:
pred_val=np.argmax(test_pred, axis=1)

In [41]:
sub_df=pd.DataFrame(pred_val, columns=['label'])
sub_df.to_csv('sub_df.csv',index=False)